# PART 1: Reproduce Results from "Ask Me Anything: Dynamic Memory Network in NLP"

In this part of the code, I used the code from this Github repository: https://github.com/jhyuklee/dmn-pytorch. This is a Pytorch implementation of the Dynamic Memory Network introduced in the paper above.

I also used the .sh files from https://github.com/DongjunLee/dmn-tensorflow to load in the bAbI dataset as well as the GloVe pre-trained word embeddings. 

I made slight modification to *dataset.py* and *run.py*. The changes are indicated with the comment #changed in the files respectively. The changed are made to deal with version update of Pytorch. 

Note: the time it takes to run each cell is indicated on the top of the cell if it is above 5 minutes.

In [14]:
from google.colab import files
!pip install tensorboardX

     |████████████████████████████████| 204kB 11.0MB/s 


In [2]:
!git clone https://github.com/jhyuklee/dmn-pytorch 
%cd dmn-pytorch
!ls
!mkdir datasets
%cd datasets/

Cloning into 'dmn-pytorch'...
remote: Enumerating objects: 153, done.
remote: Total 153 (delta 0), reused 0 (delta 0), pack-reused 153
Receiving objects: 100% (153/153), 1.96 MiB | 8.67 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/dmn-pytorch
dataset.py  LICENSE  main.py  model.py	README.md  run.py  utils.py
/content/dmn-pytorch/datasets


In [3]:
# upload fetch_babi_data.sh and fetch_glove_data.sh 
# from Part_1_reproduce_results folder
files.upload()

Saving fetch_babi_data.sh to fetch_babi_data.sh
Saving fetch_glove_data.sh to fetch_glove_data.sh


{'fetch_babi_data.sh': b'#!/bin/bash\n\nurl=http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\nfname=`basename $url`\n\ncurl -SLO $url\ntar zxvf $fname \nmkdir -p data\nmv tasks_1-20_v1-2/* data/\n\nrm -r tasks_1-20_v1-2\nrm tasks_1-20_v1-2.tar.gz\n',
 'fetch_glove_data.sh': b'#!/bin/bash\n\nurl=http://nlp.stanford.edu/data/glove.6B.zip\nfname=`basename $url`\n\ncurl -SLO $url\nmkdir -p data\nunzip $fname -d data/glove/\n'}

In [4]:
# this step takes about 6 minates
%%shell 
sh fetch_babi_data.sh
sh fetch_glove_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.9M  100 14.9M    0     0  8653k      0  0:00:01  0:00:01 --:--:-- 8648k
tasks_1-20_v1-2/
tasks_1-20_v1-2/hn/
tasks_1-20_v1-2/hn/qa16_basic-induction_train.txt
tasks_1-20_v1-2/hn/qa13_compound-coreference_train.txt
tasks_1-20_v1-2/hn/qa13_compound-coreference_test.txt
tasks_1-20_v1-2/hn/qa14_time-reasoning_test.txt
tasks_1-20_v1-2/hn/qa5_three-arg-relations_test.txt
tasks_1-20_v1-2/hn/qa17_positional-reasoning_train.txt
tasks_1-20_v1-2/hn/qa9_simple-negation_train.txt
tasks_1-20_v1-2/hn/qa12_conjunction_train.txt
tasks_1-20_v1-2/hn/qa6_yes-no-questions_train.txt
tasks_1-20_v1-2/hn/qa2_two-supporting-facts_test.txt
tasks_1-20_v1-2/hn/qa20_agents-motivations_train.txt
tasks_1-20_v1-2/hn/qa7_counting_train.txt
tasks_1-20_v1-2/hn/qa18_size-reasoning_test.txt
tasks_1-20_v1-2/hn/qa1_single-supporting-fact_train.txt
tasks_1-20_v1-2

In [10]:
!mkdir -p babi
!mv data/en-valid-10k babi/en
!mkdir glove 
!mv glove.6B.zip glove
%cd glove 
!unzip glove.6B.zip
%cd .. 
%cd .. 
!ls

dataset.py  datasets  LICENSE  main.py	model.py  README.md  run.py  utils.py


In [11]:
!rm dataset.py
!rm run.py
# upload dataset.py & run.py
# from Part_1_reproduce_results folder
files.upload()

Saving dataset.py to dataset.py
Saving run.py to run.py


{'dataset.py': b'import numpy as np\nimport sys\nimport pprint\nimport copy\nimport pickle\nimport nltk\nimport string\nimport os\nfrom os.path import expanduser\n\nnltk.download(\'punkt\')\n\n\nclass Dataset(object):\n    def __init__(self, config):\n        self.config = config\n        self.init_settings()\n        self.init_dict()\n        self.build_word_dict(self.config.data_dir)\n        self.get_pretrained_word(self.config.word2vec_path)\n        self.process_data(self.config.data_dir)\n\n    def init_settings(self):\n        self.dataset = {}\n        self.train_ptr = 0\n        self.valid_ptr = 0\n        self.test_ptr = 0\n\n    def init_dict(self):\n        self.PAD = \'PAD\'\n        self.word2idx = {}\n        self.idx2word = {}\n        self.idx2vec = []  # pretrained\n        self.word2idx[self.PAD] = 0\n        self.idx2word[0] = self.PAD\n        self.init_word_dict = {}\n    \n    def update_word_dict(self, key):\n        if key not in self.word2idx:\n            sel

In [12]:
!ls

dataset.py  datasets  LICENSE  main.py	model.py  README.md  run.py  utils.py


In [13]:
!python dataset.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
### building word dict datasets/babi/en
init dict size 158

### loading pretrained datasets/glove/glove.6B.300d.txt
apple: 131 [-0.20842, -0.019668, 0.063981, -0.71403, -0.21181]
apple: [-0.20842, -0.019668, 0.063981, -0.71403, -0.21181]
pretrained vectors (159, 300) unk 0
dictionary change 158 to 159 159

### processing datasets/babi/en
data size 1000
max sentnum 10
max slen 72
max qlen 6

[('data_dir', 'datasets/babi/en'),
 ('word2vec_type', 6),
 ('word2vec_path', 'datasets/glove/glove.6B.300d.txt'),
 ('word_embed_dim', 300),
 ('batch_size', 32),
 ('max_sentnum',
  {1: 10,
   2: 88,
   3: 320,
   4: 2,
   5: 126,
   6: 26,
   7: 52,
   8: 58,
   9: 10,
   10: 10,
   11: 10,
   12: 10,
   13: 10,
   14: 14,
   15: 8,
   16: 9,
   17: 2,
   18: 20,
   19: 5,
   20: 12}),
 ('max_slen',
  {1: 68,
   2: 552,
   3: 1875,
   4: 16,
   5: 782,
   6: 156,
   7: 321,
   8: 358,
   9: 74,
 

In [15]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [29]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [0]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 3

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [0]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 4

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [0]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 5

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [0]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 6

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [0]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 7

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [17]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 8

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [18]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 10

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [19]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 11

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [20]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 12

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [21]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 13

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [23]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 14

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [24]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 15

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [25]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 16

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [0]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 17

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [26]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 18

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [27]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 19

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

In [28]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 20

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset
{'a_cell_hdim': 100,
 'batch_size': 32,
 'beta_cnt': 10,
 'checkpoint_dir': './results/',
 'data_dir': 'datasets/babi/en',
 'data_path': './data/babi(tmp).pkl',
 'e_cell_hdim': 100,
 'early_stop': 0,
 'epoch': 20,
 'g1_dim': 500,
 'grad_max_norm': 5,
 'lr': 0.0003,
 'lr_decay': 1.0,
 'm_cell_hdim': 100,
 'max_alen': 2,
 'max_episode': 10,
 'max_qlen': {1: 4,
              2: 5,
              3: 8,
              4: 7,
              5: 8,
              6: 6,
              7: 7,
              8: 5,
              9: 6,
              10: 6,
              11: 4,
              12: 4,
              13: 4,
              14: 7,
              15: 6,
              16: 5,
              17: 12,
              18: 10,
              19: 11,
              20: 8},
 'max_sentnum': {1: 10,
                 2: 88,
                 3: 320,
                 4: 2,
                 5: 

# PARTII: Dynamic Memory Network with ELMo embeddings

In this part of the project, we changed the input word embeddings from GloVe to ELMo, which is a deep contexualized word representation that models both (1) complex characteristics of word use (e.g., syntax and semantics), and (2) how these uses vary across linguistic contexts. 

The ELMo word embeddings are taken from [Tensorflow Hub](https://tfhub.dev/google/elmo/2). 

We made changes to *dataset.py* and *model.py*. However, the rest of the files are the same as before. 

In [36]:
!ls

data	    datasets  main.py	__pycache__  results  utils.py
dataset.py  LICENSE   model.py	README.md    run.py


In [38]:
!rm dataset.py
!rm model.py
# upload dataset.py, model.py from folder Part_2_ELMo
files.upload()


Saving dataset.py to dataset.py
Saving model.py to model.py


{'dataset.py': b'import numpy as np\nimport sys\nimport pprint\nimport copy\nimport pickle\nimport nltk\nimport string\nimport os\nfrom os.path import expanduser\nimport tensorflow_hub as hub\nimport os\nos.environ[\'TF_CPP_MIN_LOG_LEVEL\'] = \'3\'\nimport tensorflow as tf\n\nelmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)\ninit = tf.compat.v1.global_variables_initializer()\nnltk.download(\'punkt\')\n\n\nclass Dataset(object):\n    def __init__(self, config):\n        self.config = config\n        self.init_settings()\n        self.init_dict()\n        self.build_word_dict(self.config.data_dir)\n        self.process_data(self.config.data_dir)\n\n    def init_settings(self):\n        self.dataset = {}\n        self.train_ptr = 0\n        self.valid_ptr = 0\n        self.test_ptr = 0\n\n    def init_dict(self):\n        self.PAD = \'PAD\'\n        self.word2idx = {}\n        self.idx2word = {}\n        self.idx2vec = []  # pretrained\n\n        # vector for period .\

In [47]:
# since we will only be deal with task 1 and 17 here, we create a new directory 
# called en1 and we will upload only the .txt files on task 1 and 17 from 
# datasets/babi/en to datasets/babi/en1

!mkdir datasets/babi/en1
%cd datasets/babi/en1

# upload .txt file here. 
files.upload()

%cd .. 
%cd ..
%cd .. 

/content/dmn-pytorch/datasets/babi/en1


In [49]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflow-hub

In [0]:
!python dataset.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### building word dict datasets/babi/en1
line added to sen2fileidx:  the pink rectangle is to the left of the triangle.
line added to sen2fileidx:  the triangle is to the left of the red square.
line added to sen2fileidx:  is the pink rectangle to the right of the red square?
line added to sen2fileidx:  no
line added to sen2fileidx:  is the pink rectangle to the left of the red square?
line added to sen2fileidx:  yes
line added to sen2fileidx:  is the red square to the right of the pink rectangle?
line added to sen2fileidx:  the red square is below the blue square.
line added to sen2fileidx:  the red square is to the left of the pink rectangle.
line added to sen2fileidx:  is the blue square below the pink rectangle?
line added to sen2fileidx:  is the pink rectangle to the left of the blue square?
line added to sen2fileidx:  is the blue square to the left of the pink rectangle?

In [32]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset

[QA set 1]
model parameters: 687,601

- Training Epoch 1
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/content/dmn-pytorch/model.py:142: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.out(memory))
/content/dmn-pytorch/model.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(z)
/cont

In [35]:
# Train with different number of hidden units, epochs, and QA sets
!python main.py --epoch 20 --set_num 17

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
### load dataset

[QA set 17]
model parameters: 687,601

- Training Epoch 1
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/content/dmn-pytorch/model.py:142: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.out(memory))
/content/dmn-pytorch/model.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(z)
/con